<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/no_polit_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Remove names of politicians.



In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "no_politicians"
MODEL_NAME = "no_politicians"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 2000000
NUM_EPOCHS = 30
BATCH_SIZE = 1000
MAX_SEQUENCE_LENGTH = 20


# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

In [65]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
len(train_df)

9879016

In [0]:
len(dev_df)

2292907

In [0]:
# Get a list of the politicians' names.
# I am using just the names listed in the op_name series.
polit_names = train_df['op_name']
polit_names2 = dev_df['op_name']
polit_names.drop_duplicates(inplace=True)
polit_names2.drop_duplicates(inplace=True)

full_names_list = polit_names.to_list() + polit_names2.to_list()
polit_names_list = [name for names in full_names_list for name in names.split()]
# Dedup the list again now that it's split.
new_list = []
for name in polit_names_list:
    if name.lower() not in new_list:
        new_list.append(name.lower())
polit_names_list = new_list

In [25]:
len(polit_names_list)

512

In [0]:
# Remove initials and names that are words with common meaning.
common_word_list = ['rob','blunt','king','mark','bob','jan','chuck','rod','love','sessions','buddy','mast',
                    'bill','holding','drew','waters','gene','green','jerry','young','brat','clay','bass',
                    'rice','jack','hatch','van','buck','tester','dick','g.','brown','fudge','dean','don',
                    'slaughter','banks','f.','lynch','flake','sires','robin']
new_list = []
for name in polit_names_list:
    if name not in common_word_list:
        new_list.append(name)
polit_names_list = new_list

In [0]:
# from google.colab import files
# import io
# all_polit_names = pd.Series(polit_names_list)
# all_polit_names.to_csv('polit_names_list.csv',encoding='utf-8-sig', header=True, index=False)
# files.download('polit_names_list.csv')

In [26]:
num_responses_per_post = train_df.post_id.value_counts().reset_index()
num_responses_per_post.columns = ['post_id', 'num_responses']
too_big_posts = num_responses_per_post[num_responses_per_post.num_responses > 50]
posts_to_sample = too_big_posts.post_id.values
# this gets all the rows for posts we DON'T need to sample 
new_train_df = train_df[~train_df.post_id.isin(posts_to_sample)]
# this should be true
assert(len(too_big_posts) + new_train_df.post_id.nunique() == train_df.post_id.nunique())
too_big_post_rows = train_df[train_df.post_id.isin(posts_to_sample)]
sampled_rows = too_big_post_rows.groupby('post_id').apply(lambda x: x.sample(n=50)).reset_index(drop=True)
new_train_df = pd.concat([new_train_df, sampled_rows])
new_train_df.shape

(3962284, 8)

In [27]:
start_time = time.time()
# Remove 1 and 2 word responses from training set.
# Split response string by spaces.
new_train_df['split_response'] = new_train_df['response_text'].str.split()
# Remove the rows with split responses that have length 1 or 2.
train_df = new_train_df[new_train_df['split_response'].str.len() > 2]
train_df.head()
print("seconds to complete:",(time.time()-start_time))
train_df.shape

seconds to complete: 39.6574432849884


(3576863, 9)

In [0]:
from string import punctuation

In [22]:
# test_df = train_df.head(10)
# test_df = test_df.copy()
# def replace_polit_names(response_list,names):
#     response_list = ' '.join('politname' if i.strip(punctuation) in names else i for i in response_list.split())
#     return response_list
# start_time = time.time()
# test_df['response_text'] = test_df['response_text'].astype(str).apply(replace_polit_names,args=(polit_names_list,))
# print("minutes to complete:",(time.time()-start_time)/60)
# test_df.head(10)

minutes to complete: 4.4182936350504556e-05


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category
0,0,57265377,M,0,Jerry,protecting birth is not the same as protecting life. you may very well pledge to the former but ...,Roger Williams,Congress_Republican
1,1,57265377,M,0,Andrea,you need to protect children and leave my body to me.,Roger Williams,Congress_Republican
2,2,57265377,M,0,Sherry,thank you,Roger Williams,Congress_Republican
3,3,57265377,M,0,Bob,thank you politname,Roger Williams,Congress_Republican
4,4,57265377,M,0,Joy,"unwanted pregnancy is a sad and unfortunate situation for anyone to find themselves in, however,...",Roger Williams,Congress_Republican
5,5,57265377,M,0,Candice,"women, the ""hosts"" of the unborn babies you are so supportive of, need access to healthcare to g...",Roger Williams,Congress_Republican
6,6,57265377,M,0,Cheri,i am pro choice and always will be,Roger Williams,Congress_Republican
7,7,57265377,M,0,Julie,thank you for taking a stand!!!!,Roger Williams,Congress_Republican
8,8,57265377,M,0,Stephen,i seem to disagree with you again mr.williams. woman have a right to determine what happens to t...,Roger Williams,Congress_Republican
9,9,57265377,M,0,Tony,please demonstrate your support by providing pre- and post-natal care to these babies and their ...,Roger Williams,Congress_Republican


In [28]:
# Remove political names from training set (takes about 30 min)
from string import punctuation
def replace_polit_names(response_list,names):
    response_list = ' '.join('politname' if i.strip(punctuation) in names else i for i in response_list.split())
    return response_list
start_time = time.time()
train_df['response_text'] = train_df['response_text'].astype(str).apply(replace_polit_names,args=(polit_names_list,))
print("minutes to complete:",(time.time()-start_time)/60)
train_df.head(10)

minutes to complete: 11.142263487974803


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response
0,0,57265377,M,0,Jerry,protecting birth is not the same as protecting life. you may very well pledge to the former but ...,Roger Williams,Congress_Republican,"[protecting, birth, is, not, the, same, as, protecting, life., you, may, very, well, pledge, to,..."
1,1,57265377,M,0,Andrea,you need to protect children and leave my body to me.,Roger Williams,Congress_Republican,"[you, need, to, protect, children, and, leave, my, body, to, me.]"
3,3,57265377,M,0,Bob,thank you politname,Roger Williams,Congress_Republican,"[thank, you, roger]"
4,4,57265377,M,0,Joy,"unwanted pregnancy is a sad and unfortunate situation for anyone to find themselves in, however,...",Roger Williams,Congress_Republican,"[unwanted, pregnancy, is, a, sad, and, unfortunate, situation, for, anyone, to, find, themselves..."
5,5,57265377,M,0,Candice,"women, the ""hosts"" of the unborn babies you are so supportive of, need access to healthcare to g...",Roger Williams,Congress_Republican,"[women,, the, ""hosts"", of, the, unborn, babies, you, are, so, supportive, of,, need, access, to,..."
6,6,57265377,M,0,Cheri,i am pro choice and always will be,Roger Williams,Congress_Republican,"[i, am, pro, choice, and, always, will, be]"
7,7,57265377,M,0,Julie,thank you for taking a stand!!!!,Roger Williams,Congress_Republican,"[thank, you, for, taking, a, stand!!!!]"
8,8,57265377,M,0,Stephen,i seem to disagree with you again mr.williams. woman have a right to determine what happens to t...,Roger Williams,Congress_Republican,"[i, seem, to, disagree, with, you, again, mr.williams., woman, have, a, right, to, determine, wh..."
9,9,57265377,M,0,Tony,please demonstrate your support by providing pre- and post-natal care to these babies and their ...,Roger Williams,Congress_Republican,"[please, demonstrate, your, support, by, providing, pre-, and, post-natal, care, to, these, babi..."
10,10,57265377,M,0,Maria,thank you for protecting the sanctity of precious life .,Roger Williams,Congress_Republican,"[thank, you, for, protecting, the, sanctity, of, precious, life, .]"


In [29]:
# Remove 1 and 2 word responses from dev set.
# Split response string by spaces.
dev_df['split_response'] = dev_df['response_text'].str.split()
# Remove the rows wtih split responses that have length 1 or 2.
dev_df = dev_df[dev_df['split_response'].str.len() > 2]
len(dev_df)

2041945

In [30]:
start_time = time.time()
dev_df['response_text'] = dev_df['response_text'].astype(str).apply(replace_polit_names,args=(polit_names_list,))
print("minutes to complete:",(time.time()-start_time)/60)
dev_df.head(10)

minutes to complete: 6.37832027276357


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,split_response
0,1965080,86681682,M,29909,James,thanks for passing this ridiculous legislation. https://www.youtube.com/watch?v=-y4wd8royre,Jim Banks,Congress_Republican,"[thanks, for, passing, this, ridiculous, legislation., https://www.youtube.com/watch?v=-y4wd8royre]"
1,1965081,86681682,M,29909,Martha,handsome young man on the left. 😂 mom,Jim Banks,Congress_Republican,"[handsome, young, man, on, the, left., 😂, mom]"
2,1965082,86681682,M,29911,Preston,you are sideways!,Jim Banks,Congress_Republican,"[you, are, sideways!]"
3,1965083,86681682,M,29911,Bobby,hi congressman banks,Jim Banks,Congress_Republican,"[hi, congressman, banks]"
4,1965084,86681682,M,29911,Landon,politname i love your office. the 5th floor is for winners.,Jim Banks,Congress_Republican,"[jim,, i, love, your, office., the, 5th, floor, is, for, winners.]"
6,1965086,86681682,M,29911,Sean,it has the penthouse.,Jim Banks,Congress_Republican,"[it, has, the, penthouse.]"
7,1965087,86681682,M,29911,Preston,"we are thankful for you, mr. banks!",Jim Banks,Congress_Republican,"[we, are, thankful, for, you,, mr., banks!]"
10,1965090,86681682,M,29911,Justin,looks like lots of space for activities.,Jim Banks,Congress_Republican,"[looks, like, lots, of, space, for, activities.]"
12,1965092,86681682,M,29911,Bobby,my friend tanner,Jim Banks,Congress_Republican,"[my, friend, tanner]"
13,1965093,86681682,M,29911,William,"nice office, now when are you going to call for an investigation into the possible treasonable o...",Jim Banks,Congress_Republican,"[nice, office,, now, when, are, you, going, to, call, for, an, investigation, into, the, possibl..."


In [0]:
# Remove the columns of lists.
train_df = train_df.drop(['split_response'], axis=1)
dev_df = dev_df.drop(['split_response'], axis=1)


In [32]:
import gc
gc.collect()

22

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = new_train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [27]:
train_df.op_gender.value_counts()

M    2625221
W     951632
Name: op_gender, dtype: int64

In [26]:
dev_df.op_gender.value_counts()

M    1731288
W     310657
Name: op_gender, dtype: int64

In [0]:
# Set the y values.
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
# Turn y values into 0 (female) and 1 (male).
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
# Convert y values to numpy arrays.
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(train_df.response_text.values) 
new_sentences_test = get_text_list(dev_df.response_text.values)

In [40]:
# Tokenize the sentences.
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 05 minutes, 35 seconds


In [0]:
# Pad the tokenized data so all sequences are the same length; see first cell for max length.
# Length should be the same as in shortened response length above...
from keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [43]:
# Get the glove embeddings.
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
\ [4 files][  2.1 GiB/  2.1 GiB]   90.7 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
/
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [0]:
# trying to figure out which words are empty here
# counter = 0
# empty_indexes = []
# for index, row in enumerate(embedding_matrix):
#   if sum(row) == 0:
#     empty_indexes.append(index)
#     counter += 1
#   if counter > 1000:
#     break

# for idx in empty_indexes:
#   try:
#     print(tokenizer.index_word[idx])
#   except:
#     print("No entry for {}".format(idx))


In [0]:
# hmmmm....
# nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
# nonzero_elements / vocab_size

In [46]:
embedding_matrix.shape

(598373, 100)

In [47]:
# Initialize the model.
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           59837300  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [51]:
# Run the model.
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=20,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 3962284 samples, validate on 2000000 samples
Epoch 1/20
3962284/3962284 [==============================] - 116s 29us/sample - loss: 8.2791e-04 - acc: 1.0000 - val_loss: 6.4449e-07 - val_acc: 1.0000
Epoch 2/20
3962284/3962284 [==============================] - 116s 29us/sample - loss: 1.3128e-06 - acc: 1.0000 - val_loss: 4.3882e-08 - val_acc: 1.0000
Epoch 3/20
3962284/3962284 [==============================] - 120s 30us/sample - loss: 1.0518e-07 - acc: 1.0000 - val_loss: 4.3680e-09 - val_acc: 1.0000
Epoch 4/20
3962284/3962284 [==============================] - 114s 29us/sample - loss: 1.1089e-08 - acc: 1.0000 - val_loss: 5.2528e-10 - val_acc: 1.0000
Epoch 5/20
3962284/3962284 [==============================] - 113s 29us/sample - loss: 1.5022e-09 - acc: 1.0000 - val_loss: 9.3907e-11 - val_acc: 1.0000
Epoch 6/20
3962284/3962284 [==============================] - 114s 29us/sample - loss: 3.2707e-10 - acc: 1.0000 - val_loss: 2.8364e-11 - val_acc: 1.0000
Epoch 7/20
3962284/3962284 [

In [52]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 1.0000
Testing Accuracy:  1.0000


In [53]:
# ROC AUC
from sklearn.metrics import roc_auc_score
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:,0]
auc = roc_auc_score(y_dev, yhat_probs)
print('ROC AUC:', auc)


ValueError: ignored

In [0]:
# can't get this to work
# plot_model(history)

In [0]:
preds = yhat_probs # model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [58]:
wrong_preds.op_gender.value_counts()

M    1731288
Name: op_gender, dtype: int64

In [59]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

1.0

In [60]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1212538,6671722,77657070,M,255777,Joe,"have another glass of kool-aid, ""freedumb guy"".",Mike Crapo,Congress_Republican,0.0,W
1804342,10327479,32570461,M,382365,Deborah,"do not know you, sen. politname therefore i have no idea whether you can answer this or not, but...",Tim Kaine,Congress_Democratic,0.0,W
1830247,10353384,55333293,M,383427,Toni,oh good for you politname stand strong.,Peter DeFazio,Congress_Democratic,0.0,W
129810,2285668,89989833,M,48913,Leisa,unless u have served your country on front lines or as a sniper or team seal 6 no one has a righ...,Ted Poe,Congress_Republican,0.0,W
785857,5034911,46506154,M,160605,Jeff,"you came prepared the other night, sir. shout out back from a regular guy in tampa bay😎",Marco Rubio,Congress_Republican,0.0,W
2193870,11836609,92368226,M,433405,Michele,i am a native californian and i read about this a few weeks ago in my home town news paper . i w...,Michael McCaul,Congress_Republican,0.0,W
2244590,12503101,40220308,M,469309,Scott,"here is how unconstitutional ""earmarks"" are abused lori. latino museum panel shovels tax dollars...",Joe Barton,Congress_Republican,0.0,W
815094,5064148,46506154,M,160692,Muriel,iran should definitely be #1 then planned parenthood,Marco Rubio,Congress_Republican,0.0,W
1103205,5352259,46506154,M,161717,Taylor,politname politname is just more of the same.... unless these politicians are talking about the ...,Marco Rubio,Congress_Republican,0.0,W
185345,2362097,62246450,M,52522,Dona,disturbing to say the least. history has a way of repeating itself which is more disturbing. gas...,Ron Wyden,Congress_Democratic,0.0,W


In [61]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1212538,6671722,77657070,M,255777,Joe,"have another glass of kool-aid, ""freedumb guy"".",Mike Crapo,Congress_Republican,0.0,W
405342,3174578,58406454,M,90824,Stan,wtg rep. wahlberg!,Tim Walberg,Congress_Republican,0.0,W
629405,4878459,46506154,M,160211,James,we have had enough politname politname guarantee us voters you will terminate your run not suspe...,Marco Rubio,Congress_Republican,0.0,W
956829,5205883,46506154,M,161288,Dorothea,i was raised in an eisenhower family of atlantic highlands do gooders town council churchgoing p...,Marco Rubio,Congress_Republican,0.0,W
1645234,10126093,51495713,M,370457,Albert,thank you sir.,Richard Shelby,Congress_Republican,0.0,W
2201540,11844279,41000375,M,435223,Penny,"ms teri,I will take any and all the help i can get for my momma .right now with the way my momma...",Hal Rogers,Congress_Republican,0.0,W
90779,2208694,38794897,M,38366,Melanie,are you saying that you have no voice politname because the last time i checked this is a page f...,Sherrod Brown,Congress_Democratic,0.0,W
1064795,5313849,46506154,M,161443,Stacey,"politname 2016 & someone else that has fantastic & outstanding like politname love, bobby jindah...",Marco Rubio,Congress_Republican,0.0,W
1373438,7791732,99469231,M,296200,Barb,football?! our social security system is not providing an income to live on. it was meant the di...,John McCain,Congress_Republican,0.0,W
1629368,10077238,49028900,M,366841,Robert,if the left wing media took your comments out of context and that has not what you meant then cl...,Steve King,Congress_Republican,0.0,W


In [62]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1212538,6671722,77657070,M,255777,Joe,"have another glass of kool-aid, ""freedumb guy"".",Mike Crapo,Congress_Republican,0.0,W
956829,5205883,46506154,M,161288,Dorothea,i was raised in an eisenhower family of atlantic highlands do gooders town council churchgoing p...,Marco Rubio,Congress_Republican,0.0,W
1645234,10126093,51495713,M,370457,Albert,thank you sir.,Richard Shelby,Congress_Republican,0.0,W
2201540,11844279,41000375,M,435223,Penny,"ms teri,I will take any and all the help i can get for my momma .right now with the way my momma...",Hal Rogers,Congress_Republican,0.0,W
90779,2208694,38794897,M,38366,Melanie,are you saying that you have no voice politname because the last time i checked this is a page f...,Sherrod Brown,Congress_Democratic,0.0,W
1064795,5313849,46506154,M,161443,Stacey,"politname 2016 & someone else that has fantastic & outstanding like politname love, bobby jindah...",Marco Rubio,Congress_Republican,0.0,W
1373438,7791732,99469231,M,296200,Barb,football?! our social security system is not providing an income to live on. it was meant the di...,John McCain,Congress_Republican,0.0,W
1629368,10077238,49028900,M,366841,Robert,if the left wing media took your comments out of context and that has not what you meant then cl...,Steve King,Congress_Republican,0.0,W
1035390,5284444,46506154,M,161381,Jlds,please just please shut the fuck up. you make stupid look smart.,Marco Rubio,Congress_Republican,0.0,W
2256323,13075190,33944100,M,481757,Abdul,- democracy could not be carried on battleships; it only can be carried by mind,Robert Aderholt,Congress_Republican,0.0,W
